In [17]:
import os
import glob
mode = 'train'
root = './datasets'
datasets_name = 'monet2photo'
'%s/A' % mode

'train/A'

In [18]:
path_A = os.path.join(root, datasets_name, mode + 'A') + '/*.*'
path_A

'./datasets\\monet2photo\\trainA/*.*'

In [20]:
#获取匹配路径名的所有文件
glob.glob(path_A)

['./datasets\\monet2photo\\trainA\\00001.jpg',
 './datasets\\monet2photo\\trainA\\00003.jpg',
 './datasets\\monet2photo\\trainA\\00004.jpg',
 './datasets\\monet2photo\\trainA\\00006.jpg',
 './datasets\\monet2photo\\trainA\\00007.jpg',
 './datasets\\monet2photo\\trainA\\00008.jpg',
 './datasets\\monet2photo\\trainA\\00009.jpg',
 './datasets\\monet2photo\\trainA\\00011.jpg',
 './datasets\\monet2photo\\trainA\\00012.jpg',
 './datasets\\monet2photo\\trainA\\00014.jpg',
 './datasets\\monet2photo\\trainA\\00015.jpg',
 './datasets\\monet2photo\\trainA\\00016.jpg',
 './datasets\\monet2photo\\trainA\\00017.jpg',
 './datasets\\monet2photo\\trainA\\00018.jpg',
 './datasets\\monet2photo\\trainA\\00019.jpg',
 './datasets\\monet2photo\\trainA\\00021.jpg',
 './datasets\\monet2photo\\trainA\\00022.jpg',
 './datasets\\monet2photo\\trainA\\00023.jpg',
 './datasets\\monet2photo\\trainA\\00024.jpg',
 './datasets\\monet2photo\\trainA\\00025.jpg',
 './datasets\\monet2photo\\trainA\\00026.jpg',
 './datasets\

In [ ]:
utiles = 11

import random
import time
import datetime
import sys

from torch.autograd import Variable
import torch
from visdom import Visdom
import numpy as np

def tensor2image(tensor):
    image = 127.5*(tensor[0].cpu().float().numpy() + 1.0)
    if image.shape[0] == 1:
        image = np.tile(image, (3,1,1))
    return image.astype(np.uint8)

class Logger():
    def __init__(self, n_epochs, batches_epoch):
        self.viz = Visdom()
        self.n_epochs = n_epochs
        self.batches_epoch = batches_epoch
        self.epoch = 1
        self.batch = 1
        self.prev_time = time.time()
        self.mean_period = 0
        self.losses = {}
        self.loss_windows = {}
        self.image_windows = {}


    def log(self, losses=None, images=None):
        self.mean_period += (time.time() - self.prev_time)
        self.prev_time = time.time()

        sys.stdout.write('\rEpoch %03d/%03d [%04d/%04d] -- ' % (self.epoch, self.n_epochs, self.batch, self.batches_epoch))

        for i, loss_name in enumerate(losses.keys()):
            if loss_name not in self.losses:
                self.losses[loss_name] = losses[loss_name].data[0]
            else:
                self.losses[loss_name] += losses[loss_name].data[0]

            if (i+1) == len(losses.keys()):
                sys.stdout.write('%s: %.4f -- ' % (loss_name, self.losses[loss_name]/self.batch))
            else:
                sys.stdout.write('%s: %.4f | ' % (loss_name, self.losses[loss_name]/self.batch))

        batches_done = self.batches_epoch*(self.epoch - 1) + self.batch
        batches_left = self.batches_epoch*(self.n_epochs - self.epoch) + self.batches_epoch - self.batch
        sys.stdout.write('ETA: %s' % (datetime.timedelta(seconds=batches_left*self.mean_period/batches_done)))

        # Draw images
        for image_name, tensor in images.items():
            if image_name not in self.image_windows:
                self.image_windows[image_name] = self.viz.image(tensor2image(tensor.data), opts={'title':image_name})
            else:
                self.viz.image(tensor2image(tensor.data), win=self.image_windows[image_name], opts={'title':image_name})

        # End of epoch
        if (self.batch % self.batches_epoch) == 0:
            # Plot losses
            for loss_name, loss in self.losses.items():
                if loss_name not in self.loss_windows:
                    self.loss_windows[loss_name] = self.viz.line(X=np.array([self.epoch]), Y=np.array([loss/self.batch]),
                                                                    opts={'xlabel': 'epochs', 'ylabel': loss_name, 'title': loss_name})
                else:
                    self.viz.line(X=np.array([self.epoch]), Y=np.array([loss/self.batch]), win=self.loss_windows[loss_name], update='append')
                # Reset losses for next epoch
                self.losses[loss_name] = 0.0

            self.epoch += 1
            self.batch = 1
            sys.stdout.write('\n')
        else:
            self.batch += 1



class ReplayBuffer():
    def __init__(self, max_size=50):
        assert (max_size > 0), 'Empty buffer or trying to create a black hole. Be careful.'
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0,1) > 0.5:
                    i = random.randint(0, self.max_size-1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

class LambdaLR():
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0), "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)

In [ ]:
with open('epoch.txt', 'w') as f:
    epoch =